In [1]:
%use dataframe, kandy(0.8.0-dev-59)

In [84]:
import net.nashat.*
import net.nashat.ChunkSelectionStrategy.*

val cfgs =
    listOf(Random, PreferLater, PreferRarest)
        .map { chunkSelectionStrategy ->
            PotuzSimulationConfig(
                params = PotuzParams(
                    numberOfChunks = 20,
                    chunkSelectionStrategy = chunkSelectionStrategy,
                    rsParams = RSParams(2, true)
                ),
                peerCount = 6,
                isGodStopMode = true,
                randomSeed = 1
            )
        }

val res = PotuzSimulation.runAll(cfgs)

val resDf = res
    .normalizePotuzLoadedResults()
    .deriveExtraResults()
// Print configs overview
resDf
    .select { config }
    .flatten { all() }
    .gather { all() }.into("Param", "Value")
    .distinct()
    .groupBy { "Param"() }.values()

Complete 2/3


Param,Value
erasure,[RsX2]
numberOfChunks,[20]
chunkSelectionStrategy,"[Random, PreferLater, PreferRarest]"
nodeCount,[1000]
peerCount,[6]
isGodStopMode,[true]
randomSeed,[1]


In [86]:
val df1 = resDf
    .explode { result }
    .cast<ResultEntryExploded>()
    .select { config.chunkSelectionStrategy and result.derived.relativeRound and result.core.chunkDistribution }
    .convert { "chunkDistribution"<List<Int>>() }.with {
        it.withIndex().toDataFrame()
    }
    .explode { "chunkDistribution"() }

In [87]:
fun barsForRounds(rounds: Set<Double>) =
    df1
        .filter { relativeRound in rounds }
        .groupBy { chunkSelectionStrategy }
        .plot {
            x(chunkDistribution.index)
            y(chunkDistribution.value)

            bars { }

            facetGrid(chunkSelectionStrategy, relativeRound)
        }

In [88]:
resDf
    .explode { result }
    .myPlotGroupDeliveredPartsAndMessageTypeCounts(adjustX2 = 60) {
        config.chunkSelectionStrategy
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="xlwFyp"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"kind":"ggbunch",
"items":[{
"x":20,
"width":1560,
"y":0,
"feature_spec":{
"layers":[{
"mapping":{
"xintercept":"relativeRound",
"color":"chunkSelectionStrategy"
},
"stat":"identity",
"data":{
"chunkSelectionStrategy":["Random","PreferLater","PreferRarest"],
"relativeRound":[3.35,2.6,2.9]
},
"linetype":"dotted",
"sampling":"none",
"position":"identity",
"geom":"vline"
},{
"mapping":{
"x":"relativeRound",
"y":"doneMsgFraction",
"color":"chunkSelectionStrategy"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"size":3.0,
"color":"#000000",
"linetype":"dotted",
"sampling":"none",
"position":"identity",
"xintercept":1.0,
"geom":"vline",
"data":{
}
}],
"mapping":{
},
"data":{
"chunkSelectionStrategy":["Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest","PreferRarest"],
"doneMsgFraction":[5.005005005005005E-5,1.5015015015015014E-4,3.503503503503503E-4,7.507507507507507E-4,0.0015515515515515516,0.003153153153153153,0.006306306306306306,0.012162162162162163,0.023023023023023025,0.04064064064064064,0.06466466466466467,0.09134134134134134,0.11616616616616617,0.13833833833833833,0.15840840840840842,0.17812812812812812,0.19514514514514514,0.212012012012012,0.22772772772772773,0.24414414414414415,0.25845845845845844,0.2742242242242242,0.29034034034034034,0.3066066066066066,0.3249249249249249,0.3434934934934935,0.3612112112112112,0.3801801801801802,0.39914914

In [90]:
barsForRounds(setOf(0.5, 1.0, 1.2, 1.5, 2.0, 3.0))

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="041LlX"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"chunkSelectionStrategy":["Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","Random","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferLater","PreferL